# Set environmental variables for local development

In [ ]:
import json, os
for k, v in json.load(open("local.settings.json"))["Values"].items():
    os.environ[k] = v
from libs.openapi.clients.oneview import OneView

# Authentication Flow

In [ ]:
import httpx

session = httpx.Client()
login = session.get("https://oneview.roku.com/sso/login", follow_redirects=True)
identifier = session.post(
    "https://login.oneview.roku.com/u/login/identifier",
    params=login.url.params,
    data={
        "state": login.url.params.get("state"),
        "username": os.environ["ONEVIEW_USERNAME"],
        "js-available": "true",
        "webauthn-available": "true",
        "is-brave": "false",
        "webauthn-platform-available": "true",
        "action": "default",
    },
    follow_redirects=True,
)
session.post(
    "https://login.oneview.roku.com/u/login/password",
    params=login.url.params,
    data={
        "state": identifier.url.params.get("state"),
        "username": os.environ["ONEVIEW_USERNAME"],
        "password": os.environ["ONEVIEW_PASSWORD"],
        "action": "default",
    },
    follow_redirects=True,
)
{k: v for k, v in session.cookies.items()}

# Send request to start report generation

In [ ]:
OneView["postReportTemplateRun"](
    parameters={"report_template_uid": os.environ["ONEVIEW_REPORT_TEMPLATE_UID"]}
)

# Find the latest version of the report

In [ ]:
import pandas as pd

data = OneView["getReports"](
    parameters={
        "filter": "report_template_uid:" + os.environ["ONEVIEW_REPORT_TEMPLATE_UID"],
        "order": "-created_at",
        "limit": 10,
    }
)
pd.DataFrame(data.model_dump()["reports"])